In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

os.chdir('./project')

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


## 0. Prepare libraries

In [2]:
!pip install coloredlogs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [4]:
from trading_bot.agent import Agent

model_name = 'model_GOOG_50'
test_stock = './dataset/data/GOOG_2019.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=False)

### 2. Load test data

In [5]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2019-01-02,1045.849976
1,2019-01-03,1016.059998
2,2019-01-04,1070.709961
3,2019-01-07,1068.390015
4,2019-01-08,1076.280029


### 3. Running Eval

In [6]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2023-05-29 04:36:37 1e0b559d61a8 root[216] DEBUG switching to TensorFlow for CPU
2023-05-29 04:36:37 1e0b559d61a8 root[216] DEBUG Buy at: $1045.85


1/1 [==============================] - 3s 3s/step


2023-05-29 04:36:40 1e0b559d61a8 root[216] DEBUG Buy at: $1016.06


1/1 [==============================] - 0s 27ms/step


2023-05-29 04:36:40 1e0b559d61a8 root[216] DEBUG Buy at: $1070.71


...

2023-05-29 04:36:57 1e0b559d61a8 root[216] DEBUG Buy at: $1343.56


1/1 [==============================] - 0s 22ms/step


2023-05-29 04:36:57 1e0b559d61a8 root[216] DEBUG Buy at: $1360.40


1/1 [==============================] - 0s 22ms/step


2023-05-29 04:36:57 1e0b559d61a8 root[216] DEBUG Sell at: $1351.89 | Position: +$2.30
2023-05-29 04:36:57 1e0b559d61a8 root[216] INFO model_GOOG_50: +$438.90



## 4. Visualize

In [7]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [8]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

## 5. Other trials

In [40]:
model_name = 'model_AAPL'
test_stock = './dataset/data/AAPL.csv'
window_size = 40
debug = True

agent = Agent(window_size, reset_every=2000, strategy='t-dqn', pretrained=False)

In [41]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Close']]
# rename feature column names
df = df.rename(columns={'Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2010-08-12,35.970001
1,2010-08-13,35.585712
2,2010-08-16,35.377144
3,2010-08-17,35.995716
4,2010-08-18,36.152859


In [17]:
coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2023-05-29 04:38:51 1e0b559d61a8 root[216] DEBUG switching to TensorFlow for CPU
2023-05-29 04:38:51 1e0b559d61a8 root[216] DEBUG Buy at: $24.17


1/1 [==============================] - 0s 67ms/step


2023-05-29 04:38:51 1e0b559d61a8 root[216] DEBUG Buy at: $23.91


1/1 [==============================] - 0s 18ms/step


2023-05-29 04:38:52 1e0b559d61a8 root[216] DEBUG Buy at: $23.77


...

2023-05-29 04:41:11 1e0b559d61a8 root[216] DEBUG Buy at: $153.46


1/1 [==============================] - 0s 28ms/step


2023-05-29 04:41:11 1e0b559d61a8 root[216] DEBUG Buy at: $155.83


1/1 [==============================] - 0s 30ms/step


2023-05-29 04:41:11 1e0b559d61a8 root[216] DEBUG Buy at: $157.08
2023-05-29 04:41:11 1e0b559d61a8 root[216] INFO model_AAPL: +$16900.50



In [18]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

## 5.1 Ours

In [108]:
model_name = 'model_BTC_2021'
test_stock = './dataset/archive/BTC-2021min.csv'
window_size = 50
debug = True

agent = Agent(window_size, reset_every=1000, strategy='t-dqn"', pretrained=False)

In [109]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['date', 'close']]
# rename feature column names
df = df.rename(columns={'close': 'actual', 'date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates
print(df.isnull().any())
df.head()

date      False
actual    False
dtype: bool


,date,actual
0,2022-03-01 03:43:00,43046.58
1,2022-03-01 03:41:00,43046.58
2,2022-03-01 03:40:00,43016.03
3,2022-03-01 03:39:00,42999.44
4,2022-03-01 03:38:00,43049.46


In [110]:
def get_stock_data(stock_file):
    """Reads stock data from csv file
    """
    df = pd.read_csv(stock_file)
    return list(df['close'])

In [111]:
coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)[:int(len(df)*0.002)]
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2023-05-29 06:00:34 1e0b559d61a8 root[216] DEBUG switching to TensorFlow for CPU
2023-05-29 06:00:34 1e0b559d61a8 root[216] DEBUG Buy at: $43046.58


1/1 [==============================] - 0s 26ms/step


2023-05-29 06:00:35 1e0b559d61a8 root[216] DEBUG Sell at: $43107.67 | Position: +$61.09


1/1 [==============================] - 0s 18ms/step


2023-05-29 06:00:38 1e0b559d61a8 root[216] DEBUG Buy at: $43154.82


...

2023-05-29 06:01:51 1e0b559d61a8 root[216] DEBUG Buy at: $38315.24


1/1 [==============================] - 0s 19ms/step


2023-05-29 06:01:51 1e0b559d61a8 root[216] DEBUG Sell at: $38314.44 | Position: -$0.80


1/1 [==============================] - 0s 18ms/step


2023-05-29 06:01:52 1e0b559d61a8 root[216] INFO model_BTC_2021: +$5.75



In [112]:
chart = visualize(df.iloc[:int(len(df)*0.002)], history, title=test_stock)
chart

<ipython-input-7-09db1cc8c2ab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['position'] = position
<ipython-input-7-09db1cc8c2ab>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['action'] = actions


alt.LayerChart(...)